In [1]:
import os
import imgaug.augmenters as iaa
import cv2
import numpy as np
import pandas as pd
from keras import Sequential
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Activation
from keras.preprocessing import image
from keras.applications import mobilenet_v2
from keras import Model

# ścieżka do danych zbioru treningowego
TRAIN_LABELS_FILE = r"/home/bartek/Documents/Kurs/DL/infoshareacademy-dl/datasets/dresses/train/labels_train.txt"
# ścieżka do danych zbioru walidacyjnego
VAL_LABELS_FILE = r"/home/bartek/Documents/Kurs/DL/infoshareacademy-dl/datasets/dresses/val/labels_val.txt"
# ścieżka do danych zbioru testowego
TEST_LABELS_FILE = r"/home/bartek/Documents/Kurs/DL/infoshareacademy-dl/datasets/dresses/test/labels_test.txt"
#ścieżka do nazw kolorów
COLOR_NAMES_FILE = r"/home/bartek/Documents/Kurs/DL/infoshareacademy-dl/datasets/dresses/names.txt"

#nadawanie kolumną nazwy kolorów
color = pd.read_csv(COLOR_NAMES_FILE)
color = color.T
color_list = list(color.iloc[0])
color_list.insert(0,'beige')
color_list.insert(0,'path')

train = pd.read_csv(TRAIN_LABELS_FILE,sep=" ",names=color_list)

def aug(img):
    # sekwencja kolejnych zmian
    seq = iaa.Sequential([
        iaa.Crop(px=(0, 16)),  # crop images from each side by 0 to 16px (randomly chosen)
        iaa.Fliplr(0.5),  # horizontally flip 50% of the images
        iaa.GaussianBlur(sigma=(0, 3.0))  # blur images with a sigma of 0 to 3.0
    ])

    # wykonanie augmentacji, funkcja przyjmuje 4 wymiarowy tensor
    return seq(images=np.array([img]))[0]

def get_generator(filename, number=None):
    # załadowanie pliku txt do ramki pandas
    df = pd.read_csv(filename, delimiter=' ', names=color_list, dtype="str")
    if number:
        df = df[:number]
    # stworzenie generatora
    gen = image.ImageDataGenerator(preprocessing_function=aug)
    # folder z danymi, do którego będą doklejane ścieżki z odpowiedniej kolumny
    directory = os.path.dirname(filename)
    # stworzenie iteratora po danych z zadanymi opcjami
    return gen.flow_from_dataframe(df, directory, "path", y_col = color_list[1:], target_size=(224, 224), batch_size=32,class_mode="other")

def create_model(gen):
    # używamy gotowego Mobilenet wytrenowanego na imaganet, ale musimy pozbyć się ostatniej warstwy klasyfikującej
    # i zastąpić ją swoją, stąd include_top=False i podany pooling
    model = mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), include_top=False, pooling="avg", weights="imagenet")
    # dodajemy na koniec sieci warstwę klasyfikującą
    dense = Dense(14,activation="sigmoid")(model.output)
    # budujemy nowe model, mobilenet z naszą warstwą klasyfikującą
    model = Model(inputs=[model.input], outputs=[dense])
    # szybki optymalizator adam - pozwala na szybszy trening, dobry loss do klasyfikacji 1 z k, dodatkowo patrzymy jak zmienia się accuracy
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["categorical_accuracy"])

    return model

def test_model(model):
    # dane testowe
    test_gen = get_generator(TEST_LABELS_FILE)

    # uruchomienie ewaluacji
    results = model.compile(test_gen)
    # wypisanie w pętli, wszytskich wartości metryk wraz z ich nazwami
    for name, result in zip(model.metrics_names, results):
        print("{}: {}".format(name, result))

def main():
    train_gen = get_generator(TRAIN_LABELS_FILE, 100)

    # dane walidacyjne
    val_gen = get_generator(VAL_LABELS_FILE,100)

    # nasze callbacki
    #callbacks = get_callbacks()

    # stwórz model
    model = create_model(train_gen)
    # ładowanie wyuczonych wcześniej wag sieci
    # model.load_weights("weights.hdf5")
    # wypisz podsumowanie modelu, wyświetla użyte warstwy i liczbę parametrów do treningu
    model.summary()
    # trening, dodajemy walidację i callbacki
    model.fit_generator(train_gen, steps_per_epoch=train_gen.n // train_gen.batch_size, epochs=10,
                        validation_data=val_gen, validation_steps=val_gen.n // val_gen.batch_size)


main()

Using TensorFlow backend.


Found 100 validated image filenames.
Found 100 validated image filenames.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchN

Epoch 1/10
3/3 [==============================] - 50s 17s/step - loss: 0.5417 - categorical_accuracy: 0.1696 - val_loss: 0.5905 - val_categorical_accuracy: 0.2188
Epoch 2/10
3/3 [==============================] - 31s 10s/step - loss: 0.2112 - categorical_accuracy: 0.6250 - val_loss: 0.3433 - val_categorical_accuracy: 0.2941
Epoch 3/10
3/3 [==============================] - 17s 6s/step - loss: 0.2196 - categorical_accuracy: 0.4453 - val_loss: 0.2785 - val_categorical_accuracy: 0.3824
Epoch 4/10
3/3 [==============================] - 29s 10s/step - loss: 0.1226 - categorical_accuracy: 0.7083 - val_loss: 0.2494 - val_categorical_accuracy: 0.4412
Epoch 5/10
3/3 [==============================] - 24s 8s/step - loss: 0.1190 - categorical_accuracy: 0.7757 - val_loss: 0.2503 - val_categorical_accuracy: 0.4792
Epoch 6/10
3/3 [==============================] - 31s 10s/step - loss: 0.0752 - categorical_accuracy: 0.8438 - val_loss: 0.2619 - val_categorical_accuracy: 0.4853
Epoch 7/10
3/3 [========